In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

In [2]:
path = "./data/20210125.xlsx"

In [3]:
df_all = pd.read_excel(path)

In [4]:
df_all.head()

,SOURCE,PROD_ID,CALL_DT,CALL_MONTH,CALL_TIME,VIP_LEVEL,ONLINE_MONTH,GENDER,IF_BLACK,AGE_SEG,...,VOICE_CHRG,DATA_CHRG,CALLED_DRTN,CALLED_DRTN01,CALLED_DRTN02,CALL_COUNT,CALLING_DRTN,CALLING_DRTN01,CALLING_DRTN02,IF_TOUCH
0,HF,234205175419,2019-11-07 17:15:00,201911,NaN,4.0,133.0,2.0,N,40.0,...,37.82,29.21,160.0,215.0,172.0,156.0,368.0,589.0,265.0,N
1,HF,234205231888,2019-06-07 10:02:00,201906,NaN,4.0,128.0,1.0,N,30.0,...,-3.23,58.75,344.0,107.0,45.0,86.0,397.0,475.0,310.0,N
2,HF,234205290296,2019-11-20 15:57:00,201911,NaN,3.0,132.0,1.0,N,60.0,...,0.96,0.00,66.0,84.0,165.0,5.0,7.0,31.0,33.0,N
3,HF,234205292854,2019-11-20 15:53:00,201911,NaN,2.0,132.0,1.0,N,50.0,...,3.99,0.01,24.0,9.0,56.0,15.0,29.0,12.0,14.0,N
4,HF,234205310711,2019-07-08 13:21:00,201907,NaN,5.0,128.0,2.0,N,45.0,...,2.44,104.58,124.0,172.0,187.0,55.0,151.0,297.0,559.0,N


In [5]:
df_all.columns

Index(['SOURCE', 'PROD_ID', 'CALL_DT', 'CALL_MONTH', 'CALL_TIME', 'VIP_LEVEL',
       'ONLINE_MONTH', 'GENDER', 'IF_BLACK', 'AGE_SEG', 'IF_GROUP',
       'IF_SCHOOL', 'IF_BRD', 'IF_DBL_TRMNL', 'TRMNL_AGE', 'IF_4G_TRMNL',
       'IF_VOLTE', 'BRD_RAX', 'CARD_3G_F', 'BRD_ACTIVE_DAYS', 'IF_ITV', 'CHRG',
       'CHRG01', 'CHRG02', 'DATA_FLOW_AMNT', 'DATA_FLOW_AMNT01',
       'DATA_FLOW_AMNT02', 'CALL_DRTN', 'CALL_DRTN01', 'CALL_DRTN02',
       'DATA_FLOW_AMNT_PRV', 'DATA_FLOW_AMNT_PRV01', 'DATA_FLOW_AMNT_PRV02',
       'DATA_FLOW_AMNT_ROAM_PRV', 'DATA_FLOW_AMNT_ROAM_PRV01',
       'DATA_FLOW_AMNT_ROAM_PRV02', 'DATA_FLOW_AMNT_5G', 'DATA_FLOW_AMNT_4G',
       'SMS_COUNT', 'CALLING_NET_COUNT', 'CALL_NET_DRTN', 'CALL_NET_DRTN01',
       'CALL_NET_DRTN02', 'VOICE_CHRG', 'DATA_CHRG', 'CALLED_DRTN',
       'CALLED_DRTN01', 'CALLED_DRTN02', 'CALL_COUNT', 'CALLING_DRTN',
       'CALLING_DRTN01', 'CALLING_DRTN02', 'IF_TOUCH'],
      dtype='object')

In [14]:
new_columns = []

In [15]:
# 三个月的CHRG
df_CHRG = pd.concat([df_all.CHRG, df_all.CHRG01, df_all.CHRG02], axis=1)
new_columns.append(df_CHRG)

In [16]:
# 三个月的DOU
df_DATA_FLOW_AMNT = pd.concat([df_all.DATA_FLOW_AMNT, df_all.DATA_FLOW_AMNT01, df_all.DATA_FLOW_AMNT02], axis=1)
new_columns.append(df_DATA_FLOW_AMNT)

In [17]:
# 三个月的省内流量 
df_DATA_FLOW_AMNT_PRV = pd.concat([df_all.DATA_FLOW_AMNT_PRV, df_all.DATA_FLOW_AMNT_PRV01, df_all.DATA_FLOW_AMNT_PRV02], axis=1)
new_columns.append(df_DATA_FLOW_AMNT_PRV)

In [18]:
# 三个月的省外流量
df_DATA_FLOW_AMNT_ROAM_PRV = pd.concat([
    df_all.DATA_FLOW_AMNT_ROAM_PRV, df_all.DATA_FLOW_AMNT_ROAM_PRV01, df_all.DATA_FLOW_AMNT_ROAM_PRV02], axis=1)
new_columns.append(df_DATA_FLOW_AMNT_ROAM_PRV)

In [19]:
# 三个月的MOU
df_CALL_NET_DRTN = pd.concat([df_all.CALL_NET_DRTN, df_all.CALL_NET_DRTN01, df_all.CALL_NET_DRTN02], axis=1)
new_columns.append(df_CALL_NET_DRTN)

In [20]:
# 三个月的被叫时长
df_CALLED_DRTN = pd.concat([df_all.CALLED_DRTN, df_all.CALLED_DRTN01, df_all.CALLED_DRTN02], axis=1)
new_columns.append(df_CALLED_DRTN)

In [21]:
# 三个月的主叫时长
df_CALLING_DRTN = pd.concat([df_all.CALLING_DRTN, df_all.CALLING_DRTN01, df_all.CALLING_DRTN02], axis=1)
new_columns.append(df_CALLING_DRTN)

In [13]:
def add_feature(data, data_all):
    """
    data: 分理处需要计算的Dataframe
    data_all: 总数据，将计算结果合并到原数据中
    """
    # 列名
    colume_name = data.columns[0]
    colume_mean = colume_name + "_mean"  # 平均值
    colume_var = colume_name + "_var"    # 方差
    colume_std = colume_name + "_std"    # 标准差
    colume_sum = colume_name + "_sum"    # 和
    # 保留两位小数
    data_all[colume_mean] = data.mean(axis=1).map(lambda x: round(x, 2))
    data_all[colume_var] = data.var(axis=1).map(lambda x: round(x, 2))
    data_all[colume_std] = data.std(axis=1).map(lambda x: round(x, 2))
    data_all[colume_sum] = data.sum(axis=1).map(lambda x: round(x, 2))

In [23]:
for data in new_columns:
    add_feature(data, df_all)

In [24]:
df_all

,SOURCE,PROD_ID,CALL_DT,CALL_MONTH,CALL_TIME,VIP_LEVEL,ONLINE_MONTH,GENDER,IF_BLACK,AGE_SEG,...,CALL_NET_DRTN_std,CALL_NET_DRTN_sum,CALLED_DRTN_mean,CALLED_DRTN_var,CALLED_DRTN_std,CALLED_DRTN_sum,CALLING_DRTN_mean,CALLING_DRTN_var,CALLING_DRTN_std,CALLING_DRTN_sum
0,HF,234205175419,2019-11-07 17:15:00,201911,NaN,4.0,133.0,2.0,N,40.0,...,7884.75,27973.0,182.33,836.33,28.92,547.0,407.33,27404.33,165.54,1222.0
1,HF,234205231888,2019-06-07 10:02:00,201906,NaN,4.0,128.0,1.0,N,30.0,...,829.65,9990.0,165.33,24902.33,157.80,496.0,394.00,6813.00,82.54,1182.0
2,HF,234205290296,2019-11-20 15:57:00,201911,NaN,3.0,132.0,1.0,N,60.0,...,371.03,1955.0,105.00,2781.00,52.74,315.0,23.67,209.33,14.47,71.0
3,HF,234205292854,2019-11-20 15:53:00,201911,NaN,2.0,132.0,1.0,N,50.0,...,479.57,1696.0,29.67,576.33,24.01,89.0,18.33,86.33,9.29,55.0
4,HF,234205310711,2019-07-08 13:21:00,201907,NaN,5.0,128.0,2.0,N,45.0,...,2747.76,18676.0,161.00,1083.00,32.91,483.0,335.67,42737.33,206.73,1007.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55615,HF,1062629879943,2019-11-06 10:14:00,201911,NaN,3.0,135.0,2.0,N,60.0,...,84.56,1730.0,64.33,142.33,11.93,193.0,48.67,9.33,3.06,146.0
55616,HF,1062629900455,2019-11-26 15:50:00,201911,NaN,3.0,135.0,2.0,N,70.0,...,734.04,5339.0,41.00,48.00,6.93,123.0,90.67,1902.33,43.62,272.0
55617,HF,1062630041846,2019-11-26 15:50:00,201911,NaN,5.0,135.0,1.0,N,55.0,...,3429.56,13493.0,143.00,1900.00,43.59,429.0,105.33,1849.33,43.00,316.0
55618,HF,1062630162397,2019-11-27 16:01:00,201911,NaN,3.0,134.0,2.0,N,40.0,...,991.59,8669.0,139.00,1477.00,38.43,417.0,119.00,201.00,14.18,357.0


In [25]:
df_all.to_excel("./data/feature_added_20210125.xlsx",encoding="utf-8")